# Assistants API

- Chat Completion API는 stateless 하게 동작하므로 history를 기억하지 못함.

Assistants API를 사용하면 자체 애플리케이션 내에 AI 도우미를 구축할 수 있습니다. 어시스턴트에는 instruction(지침)이 있으며 모델, 도구 및 파일을 활용하여 사용자 쿼리에 응답할 수 있습니다. Assistants API는 현재 코드 해석기, 파일 검색 및 함수 호출의 세 가지 유형의 도구를 지원합니다.  

<img src="https://i.imgur.com/lpsWewR.png" />

Assistants 플레이그라운드(https://platform.openai.com/playground?mode=assistant)를 사용 하거나 아래에 설명된 것과 같이 Assistants API의 기능을 탐색할 수 있습니다.

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

### 코드 해석기 (Code Interpreter)

Assistants가 샌드박스 실행 환경에서 Python 코드를 작성하고 실행할 수 있습니다. 이 도구는 다양한 데이터와 포맷의 파일을 처리할 수 있으며, 데이터와 그래프 이미지가 포함된 파일을 생성할 수 있습니다. 코드 해석기를 사용하면 어시스턴트가 코드를 반복적으로 실행하여 어려운 코드 및 수학 문제를 해결할 수 있습니다. 어시스턴트가 실행에 실패한 코드를 작성하면 코드 실행이 성공할 때까지 다른 코드 실행을 시도하여 이 코드를 반복할 수 있습니다.

### Step 1 : assistant 생성  

- assistant는 model , instructions및 같은 여러 매개변수를 사용하여 사용자의 메시지에 응답하도록 구성할 수 있는 엔터티를 나타냅니다.

Instruction: 어시스턴트와 모델이 어떻게 행동하거나 반응해야 하는지 지시  
Model: 모델 지정   
Tools: API는 OpenAI에서 구축하고 호스팅하는 코드 해석기 및 검색을 지원합니다.  
Functions: 함수 호출을 사용하면 Assistants API에 함수를 설명하고 인수와 함께 호출해야 하는 함수를 지능적으로 반환하도록 할 수 있습니다.  

### 이 예에서는 코드 해석기 도구가 활성화된 개인 수학 교사인 도우미를 만듭니다.  
예) code interpreter가 활성화된 개인 수학 교사 assistant를 생성하여 주어진 수학 문제를 푸는 python code 생성

https://platform.openai.com/docs/assistants/tools/code-interpreter

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

In [3]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="당신은 개인 수학 교사입니다. 수학 문제를 풀기 위해 코드를 작성하고 실행 시키세요.",
    tools=[{"type": "code_interpreter"}],
    model=Model
)
assistant

Assistant(id='asst_0N7ieLjLZ2CnbxkKbRibCYkm', created_at=1730530400, description=None, instructions='당신은 개인 수학 교사입니다. 수학 문제를 풀기 위해 코드를 작성하고 실행 시키세요.', metadata={}, model='gpt-4o-mini', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [4]:
print(assistant.id)
print(assistant.name)
print(assistant.instructions)
print(assistant.tools)

asst_0N7ieLjLZ2CnbxkKbRibCYkm
Math Tutor
당신은 개인 수학 교사입니다. 수학 문제를 풀기 위해 코드를 작성하고 실행 시키세요.
[CodeInterpreterTool(type='code_interpreter')]


### Step 2 : Thread 생성

- 스레드는 사용자와 하나 이상의 Assistant 간의 대화를 나타냅니다. 사용자(또는 AI 애플리케이션)가 어시스턴트와 대화를 시작할 때 스레드를 생성할 수 있습니다.  
사용자가 대화를 시작하자마자 사용자당 하나의 스레드를 생성하는 것이 좋습니다. 메시지를 생성하여 이 스레드에 사용자별 컨텍스트와 파일을 전달합니다.   

- Assistant와 Thread는 독립적으로 존재 합니다.

In [5]:
thread = client.beta.threads.create()
thread

Thread(id='thread_rXSuRmiNSDSw7ckkvCl6qGmk', created_at=1730530401, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [6]:
thread.id

'thread_rXSuRmiNSDSw7ckkvCl6qGmk'

### Step 3 : Thread에 message 추가

사용자나 애플리케이션이 생성한 메시지의 내용은 스레드에 메시지 개체로 추가됩니다. 메시지에는 텍스트와 파일이 모두 포함될 수 있습니다. 스레드에 추가할 수 있는 메시지 수에는 제한이 없습니다. 모델의 컨텍스트 창에 맞지 ​​않는 모든 컨텍스트를 스마트하게 자릅니다. 어시스턴트 API를 사용하면 특정 RUN에 대해 모델에 전달되는 입력 토큰 수에 대한 제어를 위임합니다. 즉, 경우에 따라 어시스턴트 실행 비용에 대한 제어 권한이 줄어들지만 복잡성을 처리할 필요는 없습니다. 

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    #content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
    content="방정식 `3x + 11 = 14`를 풀려고해. 도와줘."
)

message

Message(id='msg_RmhePeyNbuacTp08ga5LjB6f', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='방정식 `3x + 11 = 14`를 풀려고해. 도와줘.'), type='text')], created_at=1730530403, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_rXSuRmiNSDSw7ckkvCl6qGmk')

In [8]:
# message 객체의 모든 속성과 그 값을 사전 형태로 반환
vars(message)

{'id': 'msg_RmhePeyNbuacTp08ga5LjB6f',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [TextContentBlock(text=Text(annotations=[], value='방정식 `3x + 11 = 14`를 풀려고해. 도와줘.'), type='text')],
 'created_at': 1730530403,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_rXSuRmiNSDSw7ckkvCl6qGmk',
 '_request_id': 'req_0625a7d37dbabf3eb902e62c09f6e6fc',
 '__exclude_fields__': {'__exclude_fields__', '_request_id'}}

### Step 4 : Run 생성

모든 사용자 메시지가 스레드에 추가되면 특정 Assistant를 사용하여 스레드를 Run할 수 있습니다. Run을 생성하면 Assistant와 관련된 모델 및 도구를 사용하여 응답을 생성합니다. 이러한 응답은 스레드에 assistant메시지로 추가됩니다.

### 방법 1 - streaming 사용
OpenAI의 AssistantEventHandler를 사용하여 Run을 만들고 응답을 스트리밍할 수 있습니다. --> 동기적 방식

In [9]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# 먼저, 이벤트 핸들러 클래스(EventHandler)를 생성하여
# 응답 스트림에서 이벤트를 어떻게 처리할지 정의합니다.
class EventHandler(AssistantEventHandler):    
  # 텍스트 생성이 완료되었을 때 호출되는 메서드
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  # 텍스트 생성 중간에 호출되는 메서드
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  # 도구 호출이 생성되었을 때 호출되는 메서드
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  # 도구 호출 중간에 호출되는 메서드
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

In [10]:
event_handler = EventHandler()

# 그런 다음, `stream` SDK 도우미와 `EventHandler` 클래스를 사용하여
# Run을 생성하고 응답을 스트리밍합니다.
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",  # 어시스턴트에게 제공할 지시사항
  event_handler=event_handler,  # EventHandler 클래스 인스턴스를 이벤트 핸들러로 사용
) as stream:
  stream.until_done()  # 스트림이 완료될 때까지 대기


assistant > code_interpreter

import sympy as sp

# 방정식 정의
x = sp.symbols('x')
equation = 3*x + 11 - 14

# 방정식을 푼다
solution = sp.solve(equation, x)
solution[0]
assistant > 방정식 \( 3x + 11 = 14 \)의 해는 \( x = 1 \)입니다. 도움이 더 필요하시면 말씀해 주세요, 고객님!

### 방법 2 - streaming 사용 않음
RUN은 비동기식입니다. 즉, 터미널 상태에 도달할 때까지 Run 개체를 폴링하여 상태를 모니터링해야 한다는 뜻입니다. 'create_and_poll' SDK는 Run 생성과 완료 폴링을 모두 지원합니다.

In [11]:
# `create_and_poll` 메서드를 사용하여 새로운 실행(run)을 생성하고 해당 실행이 완료될 때까지 폴링합니다.
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

vars(run)

{'id': 'run_kuNT4manhHRBZs2sgkat3vOq',
 'assistant_id': 'asst_0N7ieLjLZ2CnbxkKbRibCYkm',
 'cancelled_at': None,
 'completed_at': 1730530420,
 'created_at': 1730530418,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1730530419,
 'status': 'completed',
 'thread_id': 'thread_rXSuRmiNSDSw7ckkvCl6qGmk',
 'tool_choice': 'auto',
 'tools': [CodeInterpreterTool(type='code_interpreter')],
 'truncation_strategy': TruncationStrategy(type='auto', last_messages=None),
 'usage': Usage(completion_tokens=24, prompt_tokens=275, total_tokens=299),
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {},
 '_request_id': 'req_182771e1027d991c1466ba2e67b147fe',
 '__exclude_fiel

In [12]:
# 실행 상태 확인
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_6NKrQ01vZr2tCqhwDhD9Zqzt', assistant_id='asst_0N7ieLjLZ2CnbxkKbRibCYkm', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='추가적인 도움이 필요하시거나 다른 질문이 있으시면 언제든지 말씀해 주세요. 고객님!'), type='text')], created_at=1730530420, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_kuNT4manhHRBZs2sgkat3vOq', status=None, thread_id='thread_rXSuRmiNSDSw7ckkvCl6qGmk'), Message(id='msg_beC9YdWXTQUl9iyEE9WHYEyL', assistant_id='asst_0N7ieLjLZ2CnbxkKbRibCYkm', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='방정식 \\( 3x + 11 = 14 \\)의 해는 \\( x = 1 \\)입니다. 도움이 더 필요하시면 말씀해 주세요, 고객님!'), type='text')], created_at=1730530417, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_VY1WsxYR6Aqz44xr2yLhxj4m', status=None, thread_id='thread_rXSuRmiNSDSw7ckkvCl6qGmk'), Mess

In [13]:
messages.data[0].role

'assistant'

In [14]:
messages.data[0].content[0].text.value

'추가적인 도움이 필요하시거나 다른 질문이 있으시면 언제든지 말씀해 주세요. 고객님!'

In [16]:
# messages 객체의 데이터에서 role과 content 값을 추출하여 리스트에 저장
values = [(messages.data[i].role, messages.data[i].content[0].text.value) for i in range(len(messages.data)-1, -1, -1)]

# 추출한 값들을 출력
for role, value in values:
    print(f"role: {role}")  # 역할(role) 출력
    print(value)  # 내용(value) 출력
    print(150*"-")  # 구분선 출력

role: user
방정식 `3x + 11 = 14`를 풀려고해. 도와줘.
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
방정식 \( 3x + 11 = 14 \)의 해는 \( x = 1 \)입니다. 도움이 더 필요하시면 말씀해 주세요, 고객님!
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
추가적인 도움이 필요하시거나 다른 질문이 있으시면 언제든지 말씀해 주세요. 고객님!
------------------------------------------------------------------------------------------------------------------------------------------------------


## Code Interpreter에 파일 전달
Assistant 에 전달된 파일은 이 Assistant를 사용하는 모든 Run에서 액세스할 수 있습니다.  

다음은 OpenAI의 code interpreter 도구를 사용하여 업로드된 mydata.csv 파일을 바탕으로 사용자에게 응답하는 샘플 코드입니다. 이 코드는 사용자가 특정 질문을 할 때 CSV 파일을 읽고, 해당 질문에 맞는 응답을 제공하는 예제입니다.

가정: 사용자가 학생의 이름을 입력하면 해당 학생의 나이와 성적을 반환하는 예제

#### 1. CSV 파일 업로드 및 어시스턴트 생성 코드

In [17]:
import pandas as pd

# 예시 데이터 생성
data = {
    "name": ["Alice", "Bob", "Charlie"],
    "age": [14, 15, 13],
    "grade": ["A", "B", "A"]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
csv_file_path = "data/mydata.csv"
df.to_csv(csv_file_path, index=False)

print(f"CSV 파일이 {csv_file_path}에 저장되었습니다.")

CSV 파일이 data/mydata.csv에 저장되었습니다.


In [71]:
# "mydata.csv" 파일을 업로드
file = client.files.create(
  file=open("data/mydata.csv", "rb"),
  purpose='assistants'
)
file

FileObject(id='file-zHiYN4Jg2kLpwzea345GF9g8', bytes=52, created_at=1730531747, filename='mydata.csv', object='file', purpose='assistants', status='processed', status_details=None)

In [72]:
# 파일 ID를 사용하여 어시스턴트 생성
assistant = client.beta.assistants.create(
  instructions="당신은 개인 비서입니다. 학생 정보에 관해 질문을 받으면 코드 해석기를 사용하여 CSV 파일에서 세부정보를 가져옵니다. 한국어로 답합니다.",
  model=Model,
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

print("어시스턴트가 생성되었습니다.")

어시스턴트가 생성되었습니다.


#### 2. 사용자 질문에 응답하는 코드 인터프리터 코드

In [73]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Alice 의 나이와 학년은 어떻게 되나요?",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [74]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

In [75]:
# 실행 상태 확인
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_CmOt1IMRqeFNmS70b90O0zPd', assistant_id='asst_4oVbn6fFnuUywx3Ymp7LmDCD', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='업로드하신 파일의 내용은 CSV 형식으로, 다음과 같은 데이터가 포함되어 있습니다:\n\n```\nname,age,grade\nAlice,14,A\nBob,15,B\nCharlie,13,A\n```\n\n따라서 Alice의 나이는 14세이고, 학년은 A입니다. 추가로 궁금한 점이 있다면 말씀해 주세요!'), type='text')], created_at=1730531756, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_bcCEnwUzTBBjY6eg9ZLMYCIh', status=None, thread_id='thread_tzoi6JUgGkl7ktMYrUnWTspt'), Message(id='msg_wYxD5jIMw7bsATBgEE97aMkA', assistant_id='asst_4oVbn6fFnuUywx3Ymp7LmDCD', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='고객님, 업로드하신 파일을 확인하여 Alice의 나이와 학년 정보를 추출하겠습니다. 잠시만 기다려 주세요.'), type='text')], created_at=1730531751, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assi

In [76]:
# messages 객체의 데이터에서 role과 content 값을 추출하여 리스트에 저장
values = [(messages.data[i].role, messages.data[i].content[0].text.value) for i in range(len(messages.data)-1, -1, -1)]

# 추출한 값들을 출력
for role, value in values:
    print(f"role: {role}")  # 역할(role) 출력
    print(value)  # 내용(value) 출력
    print(150*"-")  # 구분선 출력

role: user
Alice 의 나이와 학년은 어떻게 되나요?
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
고객님, 업로드하신 파일을 확인하여 Alice의 나이와 학년 정보를 추출하겠습니다. 잠시만 기다려 주세요.
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
업로드하신 파일의 내용은 CSV 형식으로, 다음과 같은 데이터가 포함되어 있습니다:

```
name,age,grade
Alice,14,A
Bob,15,B
Charlie,13,A
```

따라서 Alice의 나이는 14세이고, 학년은 A입니다. 추가로 궁금한 점이 있다면 말씀해 주세요!
------------------------------------------------------------------------------------------------------------------------------------------------------


### Code Interpreter에서 생성된 이미지 및 파일 읽기
API의 코드 인터프리터는 이미지 다이어그램, CSV, PDF 생성과 같은 파일을 출력합니다. 생성되는 파일에는 두 가지 유형이 있습니다.

- 이미지  
- 데이터 파일(예: csvAssistant에서 생성된 데이터가 포함된 파일)
  
file_id를 읽고 Code Interpreter가 이미지를 생성하면 Assistant Message 응답 필드 에서 이 파일을 찾아 다운로드할 수 있습니다 .

In [88]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "학생정보를 나이와 학년 분포를 막대그래프로 시각화해서 이미지로 만들어줘. 이미지에는 한글이 들어가지 않도록 영문으로 만들어줘.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [89]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

In [90]:
# 실행 상태 확인
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_Kz6n92JX1hDBK7gbEMJ3b6RM', assistant_id='asst_4oVbn6fFnuUywx3Ymp7LmDCD', attachments=[Attachment(file_id='file-TFL1hFr76eVGDWRsvvAOtkGs', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=165, file_path=FilePath(file_id='file-TFL1hFr76eVGDWRsvvAOtkGs'), start_index=121, text='sandbox:/mnt/data/age_grade_distribution.png', type='file_path')], value='고객님, 학생의 나이와 학년 분포를 나타낸 막대 그래프 이미지를 생성하였습니다. 아래의 링크를 통해 이미지를 다운로드하실 수 있습니다.\n\n[Download Age and Grade Distribution Image](sandbox:/mnt/data/age_grade_distribution.png)'), type='text')], created_at=1730531888, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_X5WAvEPeR5li6xh8NRbiqJNb', status=None, thread_id='thread_6ExwGYsGKB1xhCdVFrZV1tcp'), Message(id='msg_yoBfTCcwumiJbpyvlGdrdCtc', assistant_id='asst_4oVbn6fFnuUywx3Ymp7LmDCD', at

In [91]:
# messages 객체의 데이터에서 role과 content 값을 추출하여 리스트에 저장
values = [(messages.data[i].role, messages.data[i].content[0].text.value) for i in range(len(messages.data)-1, -1, -1)]

# 추출한 값들을 출력
for role, value in values:
    print(f"role: {role}")  # 역할(role) 출력
    print(value)  # 내용(value) 출력
    print(150*"-")  # 구분선 출력

role: user
학생정보를 나이와 학년 분포를 막대그래프로 시각화해서 이미지로 만들어줘. 이미지에는 한글이 들어가지 않도록 영문으로 만들어줘.
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
고객님, 데이터는 다음과 같은 구조로 되어 있습니다:

- **name**: 학생 이름
- **age**: 학생 나이
- **grade**: 학생 학년

데이터를 통해 나이와 학년 분포를 막대 그래프로 시각화하겠습니다. 이를 위해 각 학년별 나이를 집계하여 그래프를 생성한 다음 이미지를 만들겠습니다.
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
고객님, 학생의 나이와 학년 분포를 나타낸 막대 그래프 이미지를 생성하였습니다. 아래의 링크를 통해 이미지를 다운로드하실 수 있습니다.

[Download Age and Grade Distribution Image](sandbox:/mnt/data/age_grade_distribution.png)
------------------------------------------------------------------------------------------------------------------------------------------------------


In [92]:
file_id = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_id

'file-TFL1hFr76eVGDWRsvvAOtkGs'

In [93]:
# 클라이언트 파일에서 이미지 데이터를 가져옴
image_data = client.files.content(file_id)

# 이미지 데이터를 바이트 형태로 읽음
image_data_bytes = image_data.read()

# "output" 디렉토리에 파일 생성
with open("output/my-image.png", "wb") as file:
    file.write(image_data_bytes)

### Code Interpreter의 입력 및 출력 로그

In [94]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

vars(run_steps.data[0])

{'id': 'step_IzCK1rkURYbhZu8nS6awsh4B',
 'assistant_id': 'asst_4oVbn6fFnuUywx3Ymp7LmDCD',
 'cancelled_at': None,
 'completed_at': 1730531889,
 'created_at': 1730531888,
 'expired_at': None,
 'failed_at': None,
 'last_error': None,
 'metadata': None,
 'object': 'thread.run.step',
 'run_id': 'run_X5WAvEPeR5li6xh8NRbiqJNb',
 'status': 'completed',
 'step_details': MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_Kz6n92JX1hDBK7gbEMJ3b6RM'), type='message_creation'),
 'thread_id': 'thread_6ExwGYsGKB1xhCdVFrZV1tcp',
 'type': 'message_creation',
 'usage': Usage(completion_tokens=59, prompt_tokens=1252, total_tokens=1311),
 'expires_at': None}

## 실습: 다음과 같이 수정하여 실행

Assistant - 개인 재무상담 봇  
Thread - 내 집 마련 계획  

각자의 Prompt를 작성하여 API 실행